# Bag-of-Words for Document Classification

## Data and problem statement

We consider a situation where a list of English language documents are given and a class to which each of these belongs. We would like to extract a set of features from each document (in a consistent manner) and to choose a classification model to train on these data. As always, we need to split our data into train, validation and test partition or apply k-fold cross validation. In this training we will simplify by simply splitting this documents list at random into train and test parts (0.70 train and 0.30 test).

> For this training you'll need the *uci-news-aggregator.csv* dataset from [this kaggle project](https://www.kaggle.com/uciml/news-aggregator-dataset). You'll need to download it if you don't have it already.

In [1]:
import pandas as pd
text_df = pd.read_csv("~/workspace/data/uci-news-aggregator.csv")

# Some properties of our data
print("Columns:", text_df.columns.tolist())
print("Dimensions:", text_df.shape)

text_df.groupby("CATEGORY").size()

Columns: ['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP']
Dimensions: (422419, 8)


CATEGORY
b    115967
e    152469
m     45639
t    108344
dtype: int64

Now we will limit our document dataset to two classes (b = business, e = entertainment) so that we have a binary classification problem:

In [2]:
text_df = text_df.query("CATEGORY == 'b' | CATEGORY == 'e'")
text_df.groupby("CATEGORY").size()

CATEGORY
b    115967
e    152469
dtype: int64

In [3]:
train_text = text_df.sample(frac=0.7)
test_text = text_df.drop(train_text.index)

## Basic text preprocessing with NLTK

Since language is simply too rich to define separate variable for each word form that appears in or documents, we first transform our texts as follows, and thereby reduce number of distinct words.

1. Remove all punctuation signs and digits
2. Cast all letters to lower case
3. Remove words that appear very often (thus are little informative); we use a file from a package where standard list of stop words is found
4. Apply a lematization algorithm; there are many variants and essentially we wish to map e.g. ‘be’, ‘being’, ‘am’ , ‘is’ to a single word ‘be’; likewise ‘element’, ‘elements’, ‘elementary’, ‘elemental’ should be mapped to ‘element’. However ‘news’ should not map to ‘new’ as it is a distinct word. There are many scientific articles about how to do this in English language, and off course less for other languages and the proposed solutions also vary. 

In [4]:
# Run first time to download corpora
# import nltk
# nltk.download("wordnet")

from gensim import corpora
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

import string
import re

def clean_text(text):
    # Remove punctuation and digits
    text = re.sub(r'[^\w\s]','',text)  # Only keep words and whitespace characters
    # Make lower case
    text = text.lower()
    # Split into list for further transformations
    text = text.split(" ")
    # Remove stopwords
    text = [word for word in text if word not in STOPWORDS]
    # Lemmatize and stemming words
    text = [wnl.lemmatize(word) for word in text]
    return text

for title in text_df.TITLE[0:3]:
    print(title)
    print(clean_text(title))
    print("---")

Fed official says weak data caused by weather, should not slow taper
['fed', 'official', 'say', 'weak', 'data', 'caused', 'weather', 'slow', 'taper']
---
Fed's Charles Plosser sees high bar for change in pace of tapering
['fed', 'charles', 'plosser', 'see', 'high', 'bar', 'change', 'pace', 'tapering']
---
US open: Stocks fall after Fed official hints at accelerated tapering
['open', 'stock', 'fall', 'fed', 'official', 'hint', 'accelerated', 'tapering']
---


In [5]:
bow_list = [[clean_text(x[1].TITLE), x[1].CATEGORY == 'b']
                for x in text_df.iterrows()]
print(len(bow_list))
bow_list[0:3]

268436


[[['fed',
   'official',
   'say',
   'weak',
   'data',
   'caused',
   'weather',
   'slow',
   'taper'],
  True],
 [['fed',
   'charles',
   'plosser',
   'see',
   'high',
   'bar',
   'change',
   'pace',
   'tapering'],
  True],
 [['open',
   'stock',
   'fall',
   'fed',
   'official',
   'hint',
   'accelerated',
   'tapering'],
  True]]

## Creating binary features from documents

A human typed text is a rich representation of information which essentially requires human level of intelligence to fully understand. In particular order of words, punctuation signs, and exact grammatical forms in which words are used gives many different flavours to a piece of text. However we are only interested to separate a list of documents in two piles-one of class 0 and the other of class 1.

**Simplest approach one can take is to define one binary variable for each word that appears in any of the documents that we have to work with in this problem, in such way that if that word appears in a given document, then this feature has value 1 and of it does not appear then it has value 0.**

In this approach, if a word appears more than once, then the corresponding feature still has value 1. Also, the order of the words does not influence the extracted feature values, and also punctuation signs are disregarded all together (in fact we will remove them at the beginning of our processing).

As example ‘Story was telling about a data scientist’ and ‘Data scientist was telling a story about data’ will have exactly the same feature values sequence. As inadequate as this may seem, if the goal is to find documents about data scientists then it might in fact work well enough.

In [6]:
# Distinct set of our words
word_list = list(set([word for obs in bow_list for word in obs[0]]))
len(word_list)

48782

In [ ]:
from scipy.sparse import csr_matrix
import numpy as np

total_words = len([words for doc in bow_list for words in doc[0]])

n = 0
# Preallocate for speed
word_list = [None]*total_words
col_ind = [None]*total_words
row_ind = [None]*total_words
data = [1]*total_words

# Dynamically build word list for faster searching
for row, doc in enumerate(bow_list):
    for term in doc[0]:
        try:
            word_ind = word_list.index(term) 
        except ValueError:
            word_list.append(term)
            word_ind = len(word_list) - 1
        col_ind[n] = word_ind
        row_ind[n] = row
        n += 1

size = (len(bow_list),len(word_list))
indices = (row_ind, col_ind)
word_occurence = csr_matrix((data, indices), size)
target = np.array([row[1] for row in bow_list])

## Naive Bayes Classifier with binary features

One of the most simple types of machine learning models that is suitable here is the so called Naïve Bayes model. The simplest form of this model is of the binary classification type. There we assume that each observation consists of a vector of binary valued features $X=(x_1,x_2,…,x_k)$ (each component $x_j$ is either 0 or 1 valued) and a binary target value $Y$. By the basic Bayes theorem we have that

\begin{align*} p(Y \mid x_1,x_2,...,x_k) &= \frac{p(Y)\ p(x_1,x_2,...,x_k \mid Y)}{p(x_1,x_2,...,x_k)} \\
&= \frac{p(Y)\ p(x_1,x_2,...,x_k \mid Y)}{\sum_{i=0}^1P(x_1,x_2,...,x_k\mid Y = i)\ p(Y=i)}\end{align*}

Here $X=(x_1,x_2,…,x_k)$ is a binary valued sequence of feature values for a given observation. In this case we 'only' need to estimate probabilities $P(Y=1)$, $P(Y=0)$, and $P(x_1,x_2,…,x_k|Y=0)$, $P(x_1,x_2,…,x_k|Y=1)$ for each possible binary sequence $x_1,x_2,...,x_k$ of feature values. Now for a feature vector of length $k$, there are $2^k$ distinct binary sequences of length $k$ (in each position we can have 0 or 1, so we indeed have to take k-th power of 2). So total number of parameters to estimate is $2\cdot2^k +2$ = 'waaay too many!'.

To get a feeling of this number, if we have many features, say as many as there are different words in a list of documents, of we would work with 200 different words, then $2^{200}=16^{50}$ which is a number larger then number of atoms in the universe! Also, Some of these combinations of binary values are typically very rare in our data which amounts to very low significance (certainty) of our estimate, and that translates directly to low predictive performance of our model.

## Reducing number of parameters

A way to mitigate a too high number of parameters to estimate is the so called Naïve Bayes assumption.  We thus assume that

$$ P(x_1,x_2,...,x_k \mid Y=0) = \prod_{i=1}^{k}P(x_i \mid Y=0)$$

and

$$ P(x_1,x_2,...,x_k \mid Y=1) = \prod_{i=1}^{k}P(x_i \mid Y=1)$$

for each sequence of binary values $x_1, x_2, …, x_k$.

This greatly reduces the number of probabilities we need to estimate, now only $k$ for $p(x_1,x_2,...,x_k)$. This means a total of $2\cdot k + 2$, for a total speedup of $\frac{2\cdot k + 2}{2 \cdot 2^k + 2}$.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(0,10,1000)
plt.plot(x, (2*x + 2)/(2*2**x + 2))

We can estimate these probabilities using a maximum likelihood estimator, which gives us the following estimators:

$$p_i^0 = p(x_i=1\mid Y=0) := \frac{\#\{\text{observations with $x_i = 0$ and $y = 0$}\} + 1}{\#\{\text{observations with $Y=0$}\} + 1}$$
and
$$ p_i^1 = p(x_i=1\mid Y=1) := \frac{\#\{\text{observations with $x_i = 0$ and $y = 1$}\} + 1}{\#\{\text{observations with $Y=1$}\} + 1}.$$
and
$$ q^0 = p(Y=0) := \frac{\#\{\text{observations with $Y = 0$}\} + 1}{\#\{\text{total observations}\} + 1}.$$

Since this is binary classification, we can easily calculate the remaining probabilities as 
\begin{align*}
p(x_i = 0\mid Y=1) &= 1 - p_i^0 \\
p(x_i = 1\mid Y=1) &= 1 - p_i^1 \\
p(Y=1) &= 1-q^0
\end{align*}


## Create Term-Document matrix

## Alternative approach by looping

## Calculate probabilities using TDM